In [16]:
import os
# os.chdir('..')
import supervised 
from supervised import *

In [17]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

import pickle

In [18]:
# Read from extracted data
df = pd.read_csv('data_for_modeling_final_2022-10-28_1227.csv')

In [51]:
# # Take copy to measure data loss after clean-up
# df = data.copy()

In [52]:
# df = df.drop(columns=['mkt_carrier_fl_num', 'op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id', 'dep_delay', 'month', 'fl_date_year',
#     'fl_date_week_number', 'crs_dep_time', 'crs_arr_time', 'actual_elapsed_time'])

In [53]:
df.columns

Index(['mkt_carrier_fl_num', 'op_carrier_fl_num', 'origin_airport_id',
       'dest_airport_id', 'dep_delay', 'arr_delay', 'crs_dep_time',
       'crs_arr_time', 'crs_elapsed_time', 'actual_elapsed_time', 'air_time',
       'distance', 'month', 'fl_date_year', 'fl_date_week_number',
       'mean_dep_delay_carrier_origin_month',
       'mean_arr_delay_carrier_origin_month',
       'mean_dep_delay_carrier_origin_week',
       'mean_arr_delay_carrier_origin_week',
       'mean_dep_delay_carrier_origin_date',
       'mean_arr_delay_carrier_origin_date',
       'mean_dep_delay_carrier_origin_date_t-1_week',
       'mean_arr_delay_carrier_origin_date_t-1_week',
       'mean_dep_delay_carrier_origin_date_t-1_week_week_number',
       'mean_arr_delay_carrier_origin_date_t-1_week_week_number',
       'origin_region_Midwest', 'origin_region_Northeast',
       'origin_region_South', 'origin_region_West', 'dest_region_Midwest',
       'dest_region_Northeast', 'dest_region_South', 'dest_region_West

In [5]:
# Create list with the columns to be scaled.
# Create list with the columns to be scaled.
numeric_cols = [

# 'dep_delay', 
'arr_delay', 'year_x', 'distance_x', 
'month','crs_elapsed_time', 
'mean_payload_per_departure', 'mean_seats_per_departure', 'mean_passengers_per_departure', 
'mean_freight_per_departure', 'mean_mail_per_departure', 'mean_empty_seats_per_departure',

'mean_dep_delay_carrier_origin_month', 'mean_arr_delay_carrier_origin_month', 
'mean_dep_delay_carrier_origin_week', 'mean_arr_delay_carrier_origin_week', 
'mean_dep_delay_carrier_origin_date', 'mean_arr_delay_carrier_origin_date', 
'mean_dep_delay_carrier_origin_date_t-1_week', 'mean_arr_delay_carrier_origin_date_t-1_week', 
'mean_dep_delay_carrier_origin_date_t-1_week_week_number', 
'mean_arr_delay_carrier_origin_date_t-1_week_week_number', 
'mean_dep_delay_carrier_origin_datet-1_year_week', 
'mean_arr_delay_carrier_origin_datet-1_year_week', 
'mean_dep_delay_carrier_origin_date_t-1_year_month', 
'mean_arr_delay_carrier_origin_date_t-1_year_month'
]


# Create a list with the dummy/categorical variables:
cat_cols = [

'origin_region_Midwest', 'origin_region_Northeast',
'origin_region_South', 'origin_region_West', 'dest_region_Midwest',
'dest_region_Northeast', 'dest_region_South', 'dest_region_West',

'arr_hrs_ctg_Afternoon', 'arr_hrs_ctg_Evening', 'arr_hrs_ctg_Morning',
'arr_hrs_ctg_Night', 

'dep_hrs_ctg_Afternoon', 'dep_hrs_ctg_Evening', 'dep_hrs_ctg_Morning', 
'dep_hrs_ctg_Night',

'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 
'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 
'day_of_week_Wednesday',

'haul_length_long', 'haul_length_medium', 'haul_length_short'

       ]

In [6]:
# 2022-10-27 8:57 Here is the function to scale the data. Please read the function documentation. 
# This function has been saved to the midterm_functions.py file.
def scale_data(df, numeric_cols, cat_cols):
    """
    - Perform standardization (StandardScaler) on the numeric_cols of the dataframe. 
    - combines both numeric and categorical back to the entire feature dataframe.

    Params:
    - df: Dataframe object with both feature and target data. 
    - numeric_cols: Name of the numeric columns to be scaled.
    - cat_cols: Name of the categorical columns (dummy variables) NOT to be scaled.

    Returns: Dataframe with numeric data scaled and categorical data as-is.

    """
    # Create the scaler based on the training dataset
    scaler = StandardScaler()
    scaler.fit(df[numeric_cols])
    X_numeric_scaled = scaler.transform(df[numeric_cols])
    X_categorical = df[cat_cols].to_numpy()
    X = pd.DataFrame(np.hstack((X_categorical, X_numeric_scaled)), columns=cat_cols + numeric_cols)
    return X

In [7]:
# Call the function:
df_scaled = scale_data(df, numeric_cols, cat_cols)

In [8]:
df_scaled.columns

Index(['origin_region_Midwest', 'origin_region_Northeast',
       'origin_region_South', 'origin_region_West', 'dest_region_Midwest',
       'dest_region_Northeast', 'dest_region_South', 'dest_region_West',
       'arr_hrs_ctg_Afternoon', 'arr_hrs_ctg_Evening', 'arr_hrs_ctg_Morning',
       'arr_hrs_ctg_Night', 'dep_hrs_ctg_Afternoon', 'dep_hrs_ctg_Evening',
       'dep_hrs_ctg_Morning', 'dep_hrs_ctg_Night', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday',
       'haul_length_long', 'haul_length_medium', 'haul_length_short',
       'arr_delay', 'year_x', 'distance_x', 'month', 'crs_elapsed_time',
       'mean_payload_per_departure', 'mean_seats_per_departure',
       'mean_passengers_per_departure', 'mean_freight_per_departure',
       'mean_mail_per_departure', 'mean_empty_seats_per_departure',
       'mean_dep_delay_carrier_origin_month',
       'mean_arr_del

In [9]:
X = df_scaled.drop('arr_delay', axis = 1)   # features
y = df_scaled['arr_delay']    # labels

# Initialize the object 
lr = LinearRegression()

# Fit the model on our data
lr.fit(X, y)


print(lr.coef_)
lr.score(X,y)

[ 3.95933052e+08  3.95933052e+08  3.95933052e+08  3.95933052e+08
  6.21945124e+08  6.21945124e+08  6.21945124e+08  6.21945124e+08
 -5.60673923e+08 -5.60673923e+08 -5.60673923e+08 -5.60673923e+08
 -2.57546243e+08 -2.57546243e+08 -2.57546243e+08 -2.57546243e+08
 -8.38632083e+08 -8.38632083e+08 -8.38632083e+08 -8.38632083e+08
 -8.38632083e+08 -8.38632083e+08 -8.38632083e+08 -1.88997986e+09
 -1.88997986e+09 -1.88997986e+09 -1.39290273e+08  3.36962349e-01
 -1.93944859e-02 -4.65748107e-01  1.12200713e-02  1.82339613e+09
 -2.00634724e+09  2.46130863e-03 -3.90485307e-03 -1.54957909e+09
 -1.35129485e+10 -3.45586076e+10  2.31825100e+09  5.88171311e+09
  2.31825100e+09  5.88171311e+09  1.97326682e+09  5.76529329e+09
  2.18713629e+09  6.02320313e+09  2.27845914e+09  5.23997290e+09
  2.43758423e+09  5.76671201e+09]


0.037055738549542805

In [10]:
from sklearn.linear_model import LinearRegression

par = {
    'normalize': [True, False]
}

lr = LinearRegression()
lr_attributes = supervised(X, y, lr, par, 'Liner_Regression')
best_lr = lr_attributes.get_best_model()


**Data has been scaled.**


/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
/opt/homebrew/Caskroom/miniforge/b


Liner_Regression evaluation metrics: 
	Test data	Training data		Difference
RMSE: 		0.98		0.96		0.02
MAE: 		0.71		0.71		-0.00
R^2: 		0.04		0.04		0.00
Best model parameters from randomized search: {'normalize': True}


In [11]:
best_lr.coef_

array([ 1.34961482e+08,  1.30110097e+08,  1.83538429e+08,  1.75524537e+08,
        5.91631724e+07,  5.53728704e+07,  7.66175720e+07,  7.46155448e+07,
       -4.03470391e+08, -4.80353814e+08, -4.58449301e+08, -2.70309478e+08,
        1.36007906e+07,  1.57533871e+07,  1.64713357e+07,  4.27388451e+06,
        9.28990966e+08,  9.25878000e+08,  8.74329254e+08,  9.13320359e+08,
        9.21324494e+08,  9.17620689e+08,  9.14039310e+08, -4.12440034e+05,
       -2.08422097e+06, -2.10938442e+06,  4.23078446e+14,  3.37296196e-01,
       -1.92325417e-02, -4.66237462e-01,  1.01788594e-02,  1.55172677e+10,
       -1.70761178e+10,  3.23001520e-03, -3.81886768e-03, -1.31811008e+10,
        1.32045033e+11,  5.28500162e+11,  1.57361520e+11, -1.51541461e+11,
        1.57361520e+11, -1.51541461e+11,  8.00894962e+10, -1.00380322e+11,
       -5.45089730e+11,  3.37140551e+11, -1.95902785e+10, -4.27637075e+10,
        3.78224378e+10, -4.19413761e+11])

In [19]:
# Save the file to CSV
from datetime import datetime
datetime_now = datetime.now().strftime('%Y-%m-%d_%H%M')
filename = f'Linear_Regression_v2_{datetime_now}.pkl'
print(filename)

Linear_Regression_v2_2022-10-28_1313.pkl


In [20]:
# Save the model
pickle.dump(df,open(filename, 'wb'))

In [15]:
# Open the model and predict
pickled_model = pickle.load(open('Linear_Regression_v2_2022-10-28_1313.pkl', 'rb'))
df_test = pd.read_csv('test_prepared_2022-10-28_1131.csv')
predictions = pickled_model.predict(df_test)
print(predictions)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte